In [8]:
from google.colab import files
uploaded = files.upload()

Saving transformer_model_py_1.py to transformer_model_py_1.py


In [9]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [10]:
import numpy as np
import os
import torch
from torch import nn
#from transformer import Transformer
from transformer_model_py_1 import Transformer
from torch.utils.data import Dataset, DataLoader

In [11]:
START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

# Vocabularies
hindi_vocab = [
    START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', '।', '॥',
    'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ',
    'क', 'ख', 'ग', 'घ', 'ङ',
    'च', 'छ', 'ज', 'झ', 'ञ',
    'ट', 'ठ', 'ड', 'ढ', 'ण',
    'त', 'थ', 'द', 'ध', 'न',
    'प', 'फ', 'ब', 'भ', 'म',
    'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह',
    'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ो', 'ौ', '्', 'ॐ',
    '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', PADDING_TOKEN, END_TOKEN]

english_vocab = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                      '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                      ':', '<', '=', '>', '?', '@',
                      '[', '\\', ']', '^', '_', '`',
                      'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                      'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                      'y', 'z',
                      '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [12]:
index_eng = { i:c for i,c in enumerate(english_vocab)}
eng_index = { c:i for i,c in enumerate(english_vocab)}
index_hin = { i:c for i,c in enumerate(hindi_vocab)}
hin_index = { c:i for i,c in enumerate(hindi_vocab)}
print(len(hin_index))
print(len(eng_index))

102
71


In [13]:
eng_file = '/content/drive/MyDrive/Translator_project/train.en'
hin_file = '/content/drive/MyDrive/Translator_project/train.hi'

def read_lines(file_name):
    with open(file_name, 'r') as file:
        sentence = file.readlines()
        return sentence

eng_sentence = read_lines(eng_file)
hin_sentence = read_lines(hin_file)
print(len(eng_sentence))
# storing sentences to a list of max length
max_len = 100000 #5 percent of dataset
####1800000 should be here for full dataset
eng_sentence = eng_sentence[:max_len]
hin_sentence = hin_sentence[:max_len]

8568307


In [14]:
hin_sentence[-5:]

['भारत वेस्टइंडीज के दौरे पर चार टेस्ट मैच खेलेगी।\n',
 'हाई ब्लड प्रेशर को कम करने के साथ अनिद्रा से निजात दिलाता है।\n',
 "राष्ट्रीय स्वयं सेवक संघ (आरएसएस) के नेता इंद्रेश कुमार का मानना है कि 'पश्चिमी' परंरपरा वेलेंटाइंस डे रेप, अवैध बच्चा और औरतों के ख़िलाफ़ हिंसा का लिए ज़िम्मेदार है.\n",
 'पुलिस अधिकारी ने कहा कि इसके बारे में पूरी जानकारी पोस्टमार्टम की रिपोर्ट आने के बाद ही पता चल पायेगी।\n',
 'आईटी ग्रिड ऑफिस से मिले हार्ड डिस्कों के विश्लेषण को दौरान तेलंगाना स्टेट फॉरेंसिक साइंस लैबोरेटरी ने कहा कि कंपनी के पास तेलंगाना और आंध्र प्रदेश के 78,221,397 निवासियों के आधार से संबंधित आंकड़े हैं।\n']

In [15]:
eng_sentence = [sentence.rstrip('\n').lower() for sentence in eng_sentence]
hin_sentence = [sentence.rstrip('\n').lower() for sentence in hin_sentence]
print(len(hin_sentence))

100000


In [16]:
eng_sentence[-5:]

['india will play 4 tests against west indies',
 'reduces high blood pressure and relieves insomnia.',
 '"if rss leader indresh kumar is to be believed, the ""western"" tradition of valentine\'s day is responsible for rape, illegitimate children, and violence against women."',
 'police officials said that the details could be revealed after receiving the post-mortem report.',
 'during analysis of hard disks recovered earlier from the it grids office, telangana state forensic science laboratory (tsfsl) experts say they found that the firm was in possession of 78,221,397 records of aadhaar data belonging to telangana and andhra pradesh.']

In [17]:


def max_len(language):
    P = 99
    length = [len(i) for i in language]
    maximum_len = max(length)
    percentile =  np.percentile(length, P)
    return maximum_len , percentile

print('max lenght & Percentile of English Sentences:', max_len(eng_sentence))
print('max lenght & Percentile of Hindi Sentences:', max_len(hin_sentence))

max lenght & Percentile of English Sentences: (1363, 353.0)
max lenght & Percentile of Hindi Sentences: (1584, 354.0)


In [18]:
max_sequence_length = 360

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1)

valid_sentence_len = []
for index in range(len(hin_sentence)):
    hindi_sentence, english_sentence = hin_sentence[index], eng_sentence[index]
    if is_valid_length(hindi_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(hindi_sentence, hindi_vocab):
        valid_sentence_len.append(index)

print(f"Number of sentences: {len(hin_sentence)}")
print(f"Number of valid sentences: {len(valid_sentence_len)}")

Number of sentences: 100000
Number of valid sentences: 8954


In [19]:
valid_sentence_len

[7,
 9,
 23,
 31,
 35,
 46,
 52,
 58,
 71,
 77,
 78,
 92,
 93,
 94,
 102,
 120,
 129,
 132,
 138,
 155,
 172,
 186,
 216,
 243,
 249,
 266,
 282,
 283,
 297,
 309,
 326,
 340,
 348,
 362,
 377,
 388,
 424,
 429,
 450,
 451,
 462,
 479,
 496,
 500,
 502,
 504,
 515,
 521,
 535,
 545,
 549,
 553,
 567,
 571,
 577,
 580,
 585,
 589,
 628,
 631,
 647,
 652,
 659,
 668,
 690,
 705,
 742,
 743,
 750,
 760,
 779,
 806,
 815,
 824,
 830,
 835,
 843,
 852,
 857,
 867,
 870,
 876,
 896,
 900,
 929,
 937,
 945,
 949,
 968,
 971,
 972,
 979,
 982,
 985,
 992,
 996,
 1001,
 1024,
 1065,
 1071,
 1078,
 1108,
 1122,
 1127,
 1133,
 1134,
 1135,
 1141,
 1142,
 1149,
 1152,
 1157,
 1163,
 1170,
 1175,
 1176,
 1179,
 1183,
 1196,
 1230,
 1244,
 1246,
 1253,
 1256,
 1275,
 1279,
 1282,
 1315,
 1324,
 1329,
 1337,
 1342,
 1355,
 1360,
 1366,
 1377,
 1381,
 1382,
 1391,
 1411,
 1421,
 1431,
 1433,
 1450,
 1476,
 1490,
 1491,
 1494,
 1498,
 1511,
 1513,
 1520,
 1532,
 1536,
 1545,
 1547,
 1562,
 1564,
 1588,

In [20]:
hindi_sentences = [hin_sentence[i] for i in valid_sentence_len]
english_sentences = [eng_sentence[i] for i in valid_sentence_len]
print(hindi_sentences[0])
print(english_sentences[0])

पुलिस मौके पर मौजूद है।
police is present on the spot.


In [21]:
d_model = 512
batch_size = 30
#ffn_hidden is the number of  neurons in the hidden layer of Feed Forward Neural Network (Fully Connected)
ffn_hidden = 2048
num_head = 8
drop_prob = 0.1
num_layers = 1
# number of layers must be increased for accuracy
max_sequence_length = 360
d_output = len(hindi_vocab)


transformer = Transformer(d_model,
                        ffn_hidden,
                        num_head,
                         drop_prob,
                         num_layers,
                         max_sequence_length,
                         d_output,
                          eng_index,
                          hin_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [22]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (positional_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionWiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding

In [23]:
class TextDataset(Dataset):
    def __init__(self, english_sentences, hindi_sentences):
        self.english_sentences = english_sentences
        self.hindi_sentences = hindi_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, index):
        return self.english_sentences[index], self.hindi_sentences[index]

In [24]:
dataset = TextDataset(english_sentences, hindi_sentences)

In [25]:
len(dataset)

8954

In [26]:
dataset[-5]

('the films director ali abbas zafar shared the image on his twitter handle.',
 'फिल्म के डायरेक्टर अली अब्बास जफर ने फिल्म का टाइटल और तस्वीर ट्विटर पर शेयर की है.')

In [27]:
batch_size = 30
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [28]:
len(iterator)

299

In [29]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num>3:
        break

[('police is present on the spot.', 'it has a battery backup of 3050mah.', 'all our life as well as all our essential being is transformed into the possession of sachchidananda.', 'new delhi, 14th may, 2010', 'up and down', 'zenan, and hadashah, and migdal-gad,', 'that was nice of him, cahill said.', 'what is jcpoa?', 'make exercise a part of your daily routine.', 'the films premise is both interesting and exciting.', 'director: shyam benegal', 'the act of avoiding the liability of taxation.', 'so thats a good thing.', 'rajeev goyal will be appointed as cfo of afs.', 'the owner of the unidentified vehicle fled from the spot after the accident.', 'fcra comes into force w.e.f may 2011', 'the primary treatment is epinephrine.', "you don 't say?", 'woman dies during childbirth', 'if you pull a knife, you have to use ii.', 'during the raid they seized several documents.', 'priced from rs 40.', 'hes just a child.', 'forgive me, ma.', "pm modi's advice", 'i love playing tennis.', 'deserves pr

In [30]:
# to prevent the padding tokens from contributing to the loss.
criterian = nn.CrossEntropyLoss( ignore_index = hin_index[PADDING_TOKEN]) #so that the loss calculation ignores cases where the target label is the padding token.

for params in transformer.parameters():
    if params.dim() > 1 : # initializing weight parameters for 2d matrix or more
        nn.init.xavier_uniform(params)
        # nn.init.xavier_uniform --> initializes the weights of the layer with values drawn from a uniform distribution within a specified range.
        #  It helps in better weight initialization, which can lead to more stable training and better convergence of neural network models.

optim = torch.optim.Adam(transformer.parameters(), lr = 1e-4) # Adam Optimizer


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

<ipython-input-30-37c934e5bce7>:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(params)


cuda


In [31]:
NEG_INFTY = -1e9

def create_mask(eng_batch, hin_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length], True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal = 1)
    encoder_padding_mask = torch.full([num_sentences,max_sequence_length, max_sequence_length], False)
    decoder_padding_mask_self_attention = torch.full([num_sentences,max_sequence_length, max_sequence_length], False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences,max_sequence_length, max_sequence_length], False)

    for idx in range(num_sentences):
        eng_sentence_length, hin_sentence_length = len(eng_batch[idx]), len(hin_batch[idx]) # length of each sentence in a batch character wise(letters)
        eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length) # indices needed to be padded for english sentences
        hin_chars_to_padding_mask = np.arange(hin_sentence_length + 1, max_sequence_length)# indices needed to be padded for hindi sentences
        # The padding mask is used to mask out certain positions in the input sequence where padding tokens are present
        encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
        encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
        decoder_padding_mask_self_attention[idx, : ,hin_chars_to_padding_mask] = True
        decoder_padding_mask_self_attention[idx, hin_chars_to_padding_mask, :] = True
        decoder_padding_mask_cross_attention[idx, : ,eng_chars_to_padding_mask] = True
        decoder_padding_mask_cross_attention[idx, hin_chars_to_padding_mask, :] = True
    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask = torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask


In [32]:
from tqdm import tqdm
transformer.train()
#transformer.to(device)

total_loss = 0
num_epochs = 2
# 10+ epoch should be there for model accuracy

for epoch in range(num_epochs):
    print('Epoch', epoch)
    iterator = iter(train_loader)
    for batch_num, batch in tqdm(enumerate(iterator), total=len(iterator)):
        transformer.train()
        eng_batch, hin_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_mask(eng_batch, hin_batch)
        optim.zero_grad()
        hin_predictions = transformer(eng_batch,
                                      hin_batch,
                                      encoder_self_attention_mask,
                                      decoder_self_attention_mask,
                                      decoder_cross_attention_mask,
                                      encoder_start_token = False,
                                      encoder_end_token = False,
                                      decoder_start_token = True,
                                      decoder_end_token = True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(hin_batch, start_token = False, end_token = True)

        loss = criterian(hin_predictions.view(-1, d_output),labels.view(-1)) # -1 is a place holder
        valid_indices = torch.where(labels.view(-1) == hin_index[PADDING_TOKEN], False, True) # <-- Binary Tensor
        loss = loss.sum()/valid_indices.sum() # <-- calculating the mean loss over the valid examples.
        loss.backward()
        optim.step()
        if batch_num % 100 == 0:
            print(f'iteration {batch_num} : {loss.item()}')
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hin_batch[0]}")
            hin_sentence_predicted = torch.argmax(hin_predictions[0], axis= 1)
            predicted_sentence = ""
            for idx in hin_sentence_predicted:
                if idx == hin_index[END_TOKEN]:
                    break
                predicted_sentence += index_hin[idx.item()]
            print(f"Hindi Prediction: {predicted_sentence}")

save_path = '/content/drive/MyDrive/transformer_weights.pth'
torch.save({
    'model_state_dict': transformer.state_dict(),
    'optimizer_state_dict': optim.state_dict()
}, save_path)
print('weights saved in drive')

Epoch 0


  0%|          | 1/299 [00:13<1:07:07, 13.51s/it]

iteration 0 : 0.005238058045506477
English: police is present on the spot.
Hindi Translation: पुलिस मौके पर मौजूद है।
Hindi Prediction: ेूेैैटटेै०ैैैखईैजटणईट0ेट९य९६ै९ईओट६0६षओषईय"ऋ९ईैययईईईय९ैईईै९ई९ईई९ैै९ैै९घघईैैषैघयैेईेईझईैैैझईैैैैैयईैईईैैैाैैैैााैैैईैईईईतैमईईाईमैाईईईयैईईशययय्यईैैझ"6घघश९९९९९ैई९९९ईईईघघघैघेओ<PADDING>झझझञञघईयेैैैैैैैेैईैै<PADDING><PADDING><PADDING>ॐैैैैमईऔेईईईेऔशोङऔईेेेईझगैैैैैई<START>टट.झझझझैैैैटैटकैणणैैफैैैझैैैैैईै<START>ैैैैैैैैैै""भटैययऔ"टझझझ<START>टयैाझैैझईझञङाईऔैैैैैैईैैझऔैटैैैैैटैट०ैफिटऔ५औभदटटटैझैैफफु9<START><START>फणईैैैैैैऔिैैैै ै


 34%|███▍      | 101/299 [18:28<36:01, 10.92s/it]

iteration 100 : 0.0032528347801417112
English: cbse has also released an official notice regarding the same.
Hindi Translation: इस बाबत सीबीएसई ने आधिकारिक घोषणा कर दी है.
Hindi Prediction: ससर     क                  क       ा ा     ा        ा  ा  रा ा    र ्    ारककक 


...

iteration 100 : 0.0026950256433337927
English: cbse has also released an official notice regarding the same.
Hindi Translation: इस बाबत सीबीएसई ने आधिकारिक घोषणा कर दी है.
Hindi Prediction: इसकसा   क       क  ककासा  काकर ा  हा का कै 


 67%|██████▋   | 201/299 [36:33<17:39, 10.81s/it]

iteration 200 : 0.002319915685802698
English: due to pollution,...
Hindi Translation: प्रदूषण को देखते हुए .
Hindi Prediction: इाराार  का का  ा है  ह


100%|██████████| 299/299 [54:19<00:00, 10.90s/it]


weights saved in drive


In [33]:

checkpoint = torch.load('/content/drive/MyDrive/transformer_weights.pth')
transformer.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])
print("Model weights loaded.")


transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  hin_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_mask(eng_sentence, hin_sentence)
    predictions = transformer(eng_sentence,
                              hin_sentence,
                              encoder_self_attention_mask,
                              decoder_self_attention_mask,
                              decoder_cross_attention_mask,
                              encoder_start_token=False,
                              encoder_end_token=False,
                              decoder_start_token=True,
                              decoder_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_hin[next_token_index]
    hin_sentence = (hin_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return hin_sentence[0]

print(f"Evaluation translation {eng_sentence} : {hin_sentence}")
print("---------------------------------------------------------------------")

Model weights loaded.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [34]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [35]:
translation = translate("hi")
print(translation)

रो को कर का<END>


In [37]:
translation = translate("how are you")
print(translation)

पुर्या के है।<END>


In [41]:
import nltk

def calculate_bleu_score(reference, candidate):
    reference = [reference.split()]
    candidate = candidate.split()
    return nltk.translate.bleu_score.sentence_bleu(reference, candidate)

# Example usage
reference_translation = "आप कैसे हैं"
model_translation = "पुर्या के है"
bleu_score = calculate_bleu_score(reference_translation, model_translation)

print(f"BLEU Score: {bleu_score}")


BLEU Score: 0


In [47]:
def calculate_accuracy(reference, candidate):
    correct = sum(1 for r, c in zip(reference, candidate) if r == c)
    total = len(reference)
    return correct / total

def calculate_f1_score(reference, candidate):
    true_positive = sum(1 for r, c in zip(reference, candidate) if r == c == 1)
    false_positive = sum(1 for r, c in zip(reference, candidate) if r == 0 and c == 1)
    false_negative = sum(1 for r, c in zip(reference, candidate) if r == 1 and c == 0)

    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)

    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    return f1_score

# Example usage
reference_translation = [1, 2, 3, 4, 5]  # Replace with actual class labels
model_translation = [1, 2, 3, 4, 5]  # Replace with actual predicted class labels

accuracy = calculate_accuracy(reference_translation, model_translation)
f1_score = calculate_f1_score(reference_translation, model_translation)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1_score}")




Accuracy: 0.34
F1 Score: 0.48 
